## Import libraries


In [1]:
import requests
import re
import numpy as np
import os
import json
import time
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from adblockparser import AdblockRules
from bs4 import BeautifulSoup
import lxml
import deepl
from sklearn.model_selection import cross_val_score
from pprint import pprint
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report


## Unpickle .pkl file back to dataframe

In [2]:
data = pd.read_pickle("sources/df_articles_ap.pkl")
data['title'].replace('', np.nan, inplace=True)
data.dropna(subset=['title'], inplace=True)
data.reset_index(inplace=True, drop=True)
data

,body,id,published_at,raw_body,title,url,source.name,source_name_cred
0,Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- Pavo...,769528,2020-01-07T12:26:43.000000+0100,<p>Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- P...,Koledníci zo Šale získali na rozvojové projekt...,https://www.hlavnespravy.sk/kolednici-ziskali-...,hlavnespravy.sk,1
1,Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/AP-P...,878049,2020-04-23T06:01:14.000000+0200,<p>Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/A...,Dvaja migranti sa ľahko zranili pri streľbe v ...,https://www.hlavnespravy.sk/dvaja-migranti-sa-...,hlavnespravy.sk,1
2,Esslingen 5. februára 2018 (TASR/HSP/Foto:TASR...,726743,2018-02-05T16:46:32.000000+0100,<p>Esslingen 5. februára 2018 (TASR/HSP/Foto:T...,V Nemecku štvorčlenná rodina prišla o život zr...,https://www.hlavnespravy.sk/v-nemecku-stvorcle...,hlavnespravy.sk,1
3,\nInformační doplněk k hlavnímu proudu\nSource...,773010,2020-01-14T08:28:39.000000+0100,"<p><a href=""https://prima.iprima.cz/ranni-kavi...",14. 1. 2020 &#8211; Svět při úterní ranní kávi...,https://slobodnyvysielac.sk/2020/01/14/14-1-20...,slobodnyvysielac.sk,0
4,Bratislava 24. júla 2020 (HSP/Združenia sloven...,850448,2020-07-24T09:00:50.000000+0200,"<p><a href=""https://www.hlavnespravy.sk/rozhov...","Rozhovor s doc. Jánom Dudášom, DrSc. o jeho tr...",https://slobodnyvysielac.sk/2020/07/24/rozhovo...,slobodnyvysielac.sk,0
...,...,...,...,...,...,...,...,...
76735,Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/AP/K...,739558,2018-10-03T19:32:17.000000+0200,<p>Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/A...,Novým vietnamským prezidentom bude generálny t...,https://www.hlavnespravy.sk/novym-vietnamskym-...,hlavnespravy.sk,1
76736,Bratislava 10. marca 2020 (TASR/HSP/Foto:SITA-...,870796,2020-03-10T11:21:09.000000+0100,<p>Bratislava 10. marca 2020 (TASR/HSP/Foto:SI...,Miroslav Lajčák sa zúčastní na ministerskom st...,https://www.hlavnespravy.sk/miroslav-lajcak-sa...,hlavnespravy.sk,1
76737,Vždy a tak aj pri vzdelaní máte na výber. Buď ...,471352,2019-12-07T09:40:33.000000+0100,"<div>\n\n <p class=""perex""><str...",Škola pre mimoriadne nadané deti (a kam chodia...,https://romanhutira.blog.sme.sk/c/522598/skola...,sme.sk,1
76738,Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP-Ra...,749541,2019-04-06T16:52:48.000000+0200,<p>Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP...,Zástupcovia žltých viest z celého Francúzska r...,https://www.hlavnespravy.sk/zastupcovia-zltych...,hlavnespravy.sk,1


## Request to DEEPL for title translation to EN-US

In [ ]:
auth_key = 'b7c3ce73-a789-7481-259d-06991eba5752:fx'
translator = deepl.Translator(auth_key)
#result_list = []

for i in range(68813,76740):
    print(i)
    result_object = []
    result = translator.translate_text(data['title'][i], target_lang="EN-US")
    result_object.append(str(result))
    
    result_list.append(result_object)

#print(result_list)

In [ ]:
# textfile = open("sources/clickbaits.txt", "w")
# for i in range(len(result_list)):
#     textfile.write(str(result_list[i][0] + "\n"))
# textfile.close()

## Load ids of sentiment subset 100 ids of manually anotated articles + all features

In [3]:
#load subset of randomly choosed subset of 100 articles
with open("sources/manually_annotated_subset/subset_100_ids.txt", "r") as file:
    readline=file.read().split("\n")
    print(readline)

#load labels for articles above
with open("sources/manually_annotated_subset/clickbait_subset_100_labels.txt", "r") as file:
    labels=file.read().split("\n")
    print(labels)
    
# load features
df_features = pd.read_fwf('sources/clickbait_list.txt')
df_features

['8', '120', '17', '3529', '18', '128', '19', '7775', '45619', '2184', '134', '15385', '5464', '129', '21', '25', '29', '36', '163', '45', '134', '52', '180', '65', '3187', '74', '48798', '1420', '77', '78', '47204', '79', '84', '140', '21917', '151', '9549', '163', '19347', '33333', '66', '100', '164', '12909', '122', '133', '1284', '135', '136', '144', '167', '5337', '146', '14052', '64612', '67489', '251', '189', '152', '39310', '46', '9376', '162', '45083', '190', '197', '215', '218', '3', '225', '236', '4', '239', '11', '242', '245', '28', '247', '60706', '254', '273', '37', '299', '12217', '301', '305', '39', '22035', '37348', '51', '335', '60744', '60', '336', '337', '86', '30208', '357', '376', '112']
['0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', 

,clickbait_percentage
0,9.27
1,0.71
2,12.18
3,1.99
4,80.02
...,...
68807,53.12
68808,64.34
68809,64.39
68810,5.44


In [4]:
my_list = []
my_array = []
itter = 0

for i in readline:
    my_array.append(i)
    my_array.append(data['id'][int(i)])
    my_array.append(data['url'][int(i)])
    my_array.append(labels[itter])
    my_list.append(my_array)
    itter += 1
    
    my_array = []

df_100 = pd.DataFrame(my_list, columns =['poradie', 'id', 'url', 'labels']) 

In [5]:
data = pd.concat([data, df_features], axis=1) 

In [6]:
df_100 = pd.merge(df_100, data)

## Decision tree with gridsearch

In [8]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.model_selection import GridSearchCV


X = df_100.clickbait_percentage.values.reshape(-1,1) # Features
y = df_100.labels


#x_train,x_test,y_train,y_test = cross_validation.train_test_split(x,y,test_size=0.4,random_state=0)
depth = []
for i in range(2,10):
    clf = tree.DecisionTreeClassifier(max_depth=i)
    # Perform 5-fold cross validation 
    scores = cross_val_score(estimator=clf, X=X, y=y, cv=5, n_jobs=4)
    depth.append((i,scores.mean()))
print(depth)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

clf = DecisionTreeClassifier(criterion="entropy", max_depth=2)
clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[(2, 0.89), (3, 0.8800000000000001), (4, 0.8800000000000001), (5, 0.8800000000000001), (6, 0.85), (7, 0.8400000000000001), (8, 0.8), (9, 0.82)]
Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        19
           1       0.00      0.00      0.00         1

    accuracy                           0.95        20
   macro avg       0.47      0.50      0.49        20
weighted avg       0.90      0.95      0.93        20



/home/ws16-silhavik/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ws16-silhavik/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ws16-silhavik/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
y_pred = clf.predict(df_features)
y_pred = pd.DataFrame(y_pred, columns =['label'])
y_pred

data = pd.concat([data, y_pred], axis=1)
data

/home/ws16-silhavik/.local/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


,body,id,published_at,raw_body,title,url,source.name,source_name_cred,clickbait_percentage,label
0,Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- Pavo...,769528,2020-01-07T12:26:43.000000+0100,<p>Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- P...,Koledníci zo Šale získali na rozvojové projekt...,https://www.hlavnespravy.sk/kolednici-ziskali-...,hlavnespravy.sk,1,9.27,0
1,Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/AP-P...,878049,2020-04-23T06:01:14.000000+0200,<p>Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/A...,Dvaja migranti sa ľahko zranili pri streľbe v ...,https://www.hlavnespravy.sk/dvaja-migranti-sa-...,hlavnespravy.sk,1,0.71,1
2,Esslingen 5. februára 2018 (TASR/HSP/Foto:TASR...,726743,2018-02-05T16:46:32.000000+0100,<p>Esslingen 5. februára 2018 (TASR/HSP/Foto:T...,V Nemecku štvorčlenná rodina prišla o život zr...,https://www.hlavnespravy.sk/v-nemecku-stvorcle...,hlavnespravy.sk,1,12.18,0
3,\nInformační doplněk k hlavnímu proudu\nSource...,773010,2020-01-14T08:28:39.000000+0100,"<p><a href=""https://prima.iprima.cz/ranni-kavi...",14. 1. 2020 &#8211; Svět při úterní ranní kávi...,https://slobodnyvysielac.sk/2020/01/14/14-1-20...,slobodnyvysielac.sk,0,1.99,0
4,Bratislava 24. júla 2020 (HSP/Združenia sloven...,850448,2020-07-24T09:00:50.000000+0200,"<p><a href=""https://www.hlavnespravy.sk/rozhov...","Rozhovor s doc. Jánom Dudášom, DrSc. o jeho tr...",https://slobodnyvysielac.sk/2020/07/24/rozhovo...,slobodnyvysielac.sk,0,80.02,0
...,...,...,...,...,...,...,...,...,...,...
76735,Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/AP/K...,739558,2018-10-03T19:32:17.000000+0200,<p>Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/A...,Novým vietnamským prezidentom bude generálny t...,https://www.hlavnespravy.sk/novym-vietnamskym-...,hlavnespravy.sk,1,NaN,NaN
76736,Bratislava 10. marca 2020 (TASR/HSP/Foto:SITA-...,870796,2020-03-10T11:21:09.000000+0100,<p>Bratislava 10. marca 2020 (TASR/HSP/Foto:SI...,Miroslav Lajčák sa zúčastní na ministerskom st...,https://www.hlavnespravy.sk/miroslav-lajcak-sa...,hlavnespravy.sk,1,NaN,NaN
76737,Vždy a tak aj pri vzdelaní máte na výber. Buď ...,471352,2019-12-07T09:40:33.000000+0100,"<div>\n\n <p class=""perex""><str...",Škola pre mimoriadne nadané deti (a kam chodia...,https://romanhutira.blog.sme.sk/c/522598/skola...,sme.sk,1,NaN,NaN
76738,Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP-Ra...,749541,2019-04-06T16:52:48.000000+0200,<p>Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP...,Zástupcovia žltých viest z celého Francúzska r...,https://www.hlavnespravy.sk/zastupcovia-zltych...,hlavnespravy.sk,1,NaN,NaN


In [10]:
data.to_pickle("sources/indicators_final_dataset/clickbait.pkl") 

##TESTING
##TESTING
##TESTING

In [12]:
#show only non NaN rows
data[~data['label'].isnull()]

,body,id,published_at,raw_body,title,url,source.name,source_name_cred,clickbait_percentage,label
0,Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- Pavo...,769528,2020-01-07T12:26:43.000000+0100,<p>Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- P...,Koledníci zo Šale získali na rozvojové projekt...,https://www.hlavnespravy.sk/kolednici-ziskali-...,hlavnespravy.sk,1,9.27,0
1,Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/AP-P...,878049,2020-04-23T06:01:14.000000+0200,<p>Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/A...,Dvaja migranti sa ľahko zranili pri streľbe v ...,https://www.hlavnespravy.sk/dvaja-migranti-sa-...,hlavnespravy.sk,1,0.71,1
2,Esslingen 5. februára 2018 (TASR/HSP/Foto:TASR...,726743,2018-02-05T16:46:32.000000+0100,<p>Esslingen 5. februára 2018 (TASR/HSP/Foto:T...,V Nemecku štvorčlenná rodina prišla o život zr...,https://www.hlavnespravy.sk/v-nemecku-stvorcle...,hlavnespravy.sk,1,12.18,0
3,\nInformační doplněk k hlavnímu proudu\nSource...,773010,2020-01-14T08:28:39.000000+0100,"<p><a href=""https://prima.iprima.cz/ranni-kavi...",14. 1. 2020 &#8211; Svět při úterní ranní kávi...,https://slobodnyvysielac.sk/2020/01/14/14-1-20...,slobodnyvysielac.sk,0,1.99,0
4,Bratislava 24. júla 2020 (HSP/Združenia sloven...,850448,2020-07-24T09:00:50.000000+0200,"<p><a href=""https://www.hlavnespravy.sk/rozhov...","Rozhovor s doc. Jánom Dudášom, DrSc. o jeho tr...",https://slobodnyvysielac.sk/2020/07/24/rozhovo...,slobodnyvysielac.sk,0,80.02,0
...,...,...,...,...,...,...,...,...,...,...
68807,Chemnitz 18. septembra 2018 AKTUALIZOVANÉ (TAS...,738736,2018-09-19T01:16:05.000000+0200,<p>Chemnitz 18. septembra 2018 AKTUALIZOVANÉ (...,Iračana podozrivého zo zabitia muža v Chemnitz...,https://www.hlavnespravy.sk/podozriveho-zo-zab...,hlavnespravy.sk,1,53.12,0
68808,Bratislava 26. februára 2019 (SITA/HSP/Foto:TA...,494458,2019-02-26T13:01:20.000000+0100,<p>Bratislava 26. februára 2019 (SITA/HSP/Foto...,Bratislava bola ôsmym najvýkonnejším regiónom EÚ,https://www.hlavnespravy.sk/bratislava-bola-os...,hlavnespravy.sk,1,64.34,0
68809,"\nSpojené štáty priamo, alebo nepriamo kontrol...",447163,2019-02-14T08:12:33.000000+0100,"<p><a href=""https://slobodnyvyber.sk/najmocnej...","Najmocnejší muž na svete je John Bolton, porad...",https://slobodnyvysielac.sk/2019/02/14/najmocn...,slobodnyvysielac.sk,0,64.39,0
68810,Mounth Holly 31. augusta 2018 (SITA/AP/HSP/Fot...,737802,2018-08-31T17:48:06.000000+0200,<p>Mounth Holly 31. augusta 2018 (SITA/AP/HSP/...,V USA musí žena odovzdať peniaze vyzbierané pr...,https://www.hlavnespravy.sk/v-usa-musi-zena-od...,hlavnespravy.sk,1,5.44,0
